In [36]:
import os
import pandas as pd
import subprocess

root_dir = ""
# root_dir = "/dockerx/pytorch/scripts/amd"
data_path = os.path.join(root_dir, "20220318_Daily_Mathews_pytorch_test_audit.xlsx")
df = pd.read_excel(data_path, sheet_name="20220316_ROCm4.5.2_test1_test2", engine='openpyxl')

df = df[df["Assignee"] == "Michael Melesse"]

In [40]:

def generate_commands(test_data_df, split_commands=["TestJitCUDA"]):
    commands = {}
    for d in test_data_df.to_dict(orient="records"):
        # print(d)
        test_file = d["test file"]
        test_file_path = os.path.join("/tmp/pytorch/test", test_file)
        test_class = d["class"].replace("(", "").replace(")", "").replace("__main__.", "")
        test_name = d['test name']
        command = "python {}.py --verbose {}.{} ".format(
            test_file_path, test_class, test_name)
        if test_file not in commands:
            commands[test_file] = {test_class: [command]}
        else:
            if test_class not in commands[test_file]:
                commands[test_file][test_class] = [command]
            else:
                if test_class in split_commands:
                    commands[test_file][test_class].append(command)
                else:
                    commands[test_file][test_class][0] += "{}.{} ".format(test_class, test_name)

    return commands


def run_commands(commands):
    for file, command in commands.items():
        print("file", file)
        process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()


def write_commands(commands):
    for test_file in commands:
        for test_class in commands[test_file]:
            command = commands[test_file][test_class]
            if type(command) == str:
                command = [command]

            for c in command:
                file_name = "{}.{}.txt".format(test_file, test_class)
                if os.path.exists(file_name):
                    text_file = open(file_name, "a")
                else:
                    text_file = open(file_name, "w")

                # print(command)
                n = text_file.write(c)
                text_file.write("\n")
                text_file.close()


In [41]:
commands = generate_commands(df)
# print(commands["test_ops"]["TestJitCUDA"])
write_commands(commands)

In [39]:
tests = {}
for d in df.to_dict(orient="records"):
    test_file = d['test file']
    test_class = d['class'].replace("(", "").replace(")", "").replace("__main__.", "")
    test_name = d['test name']

    if test_file not in tests:
        tests[test_file] = {test_class: [test_name]}
    else:
        if test_class not in tests[test_file]:
            (tests[test_file])[test_class]=[test_name]
        else:
            (tests[test_file])[test_class].append(test_name)


for test_file in tests:
    print(test_file)
    for test_class in tests[test_file]:
        print('\t', test_class)
        for test in tests[test_file][test_class]:
            print('\t\t', test)


test_ops
	 TestMathBitsCUDA
		 test_conj_view_fft_fft2_cuda_complex64
		 test_conj_view_fft_fft_cuda_complex64
		 test_conj_view_fft_fftn_cuda_complex64
		 test_conj_view_fft_hfft2_cuda_complex64
		 test_conj_view_fft_hfft_cuda_complex64
		 test_conj_view_fft_hfftn_cuda_complex64
		 test_conj_view_fft_ifft2_cuda_complex64
		 test_conj_view_fft_ifft_cuda_complex64
		 test_conj_view_fft_ifftn_cuda_complex64
		 test_conj_view_fft_irfft2_cuda_complex64
		 test_conj_view_fft_irfft_cuda_complex64
		 test_conj_view_fft_irfftn_cuda_complex64
		 test_neg_conj_view_fft_fft2_cuda_complex128
		 test_neg_conj_view_fft_fft_cuda_complex128
		 test_neg_conj_view_fft_fftn_cuda_complex128
		 test_neg_conj_view_fft_hfft2_cuda_complex128
		 test_neg_conj_view_fft_hfft_cuda_complex128
		 test_neg_conj_view_fft_hfftn_cuda_complex128
		 test_neg_conj_view_fft_ifft2_cuda_complex128
		 test_neg_conj_view_fft_ifft_cuda_complex128
		 test_neg_conj_view_fft_ifftn_cuda_complex128
		 test_neg_conj_view_fft_irfft2_c